In [ ]:
# import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# create data

nPerClust = 100
blur = 1

A = [  1,  3 ]
B = [  1, -2 ]

# generate data
a = [ A[0]+np.random.randn(nPerClust)*blur , A[1]+np.random.randn(nPerClust)*blur ]
b = [ B[0]+np.random.randn(nPerClust)*blur , B[1]+np.random.randn(nPerClust)*blur ]

# true labels
labels_np = np.vstack((np.zeros((nPerClust,1)),np.ones((nPerClust,1))))

# concatanate into a matrix
data_np = np.hstack((a,b)).T

# convert to a pytorch tensor
data = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()

# show the data
fig = plt.figure(figsize=(5,5))
plt.plot(data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs')
plt.plot(data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ko')
plt.title('The qwerties!')
plt.xlabel('qwerty dimension 1')
plt.ylabel('qwerty dimension 2')
plt.show()

# Classes to build and train the model

In [ ]:
class ANNMultilayer(nn.Module):
    def __init__(self):
        super().__init__()
        # model architecture
        self._input = nn.Linear(2,16)  # input layer
        self._hidden = nn.Linear(16,1)  # hidden layer
        self._output = nn.Linear(1,1)  # output unit

    def forward(self, input):
        x = F.relu(self._input(input))
        x = F.relu(self._hidden(x))
        x = self._output(x)
        ann_result = torch.sigmoid(x)  # final activation unit
        return ann_result

In [ ]:
class ANNPipeline():
    def __init__(self, learning_rate):
        self._ann = ANNMultilayer()
        # loss function
        self._lossfun = nn.BCELoss() # but better to use BCEWithLogitsLoss
        # optimizer
        self._optimizer = torch.optim.SGD(self._ann.parameters(), lr=learning_rate)

    def train(self, numepochs=1000):
        # initialize losses
        losses = torch.zeros(numepochs)
        # loop over epochs
        for epochi in range(numepochs):
            # forward pass
            yHat = self._ann.forward(data)
            # compute loss
            loss = self._lossfun(yHat,labels)
            losses[epochi] = loss
            # backprop
            self._optimizer.zero_grad()
            loss.backward()
            self._optimizer.step()
        # final forward pass
        predictions = self._ann.forward(data)
        # compute the predictions and report accuracy
        # NOTE: Wasn't this ">0" previously?!?!
        totalacc = 100*torch.mean(((predictions > .5) == labels).float())
        return losses, predictions, totalacc

# Test the new code by running it once

In [ ]:
numepochs = 1000
# create everything
net = ANNPipeline(.01)
# run it
losses, predictions, totalacc = net.train(numepochs)
# report accuracy
print(f'Final accuracy: {totalacc}')

# show the losses
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch'), plt.ylabel('Loss')
plt.show()

# Now for the real test (varying learning rates)

In [ ]:
# learning rates
learning_rates = np.linspace(.001,.1,50)
# initialize
accByLR = []
allLosses = np.zeros((len(learning_rates), numepochs))

# the loop
for i, lr in enumerate(learning_rates):
    # create and run the model
    net = ANNPipeline(lr)
    losses, predictions, totalacc = net.train(numepochs)
    # store the results
    accByLR.append(totalacc)
    allLosses[i,:] = losses.detach()


In [ ]:
# plot the results
fig,ax = plt.subplots(1,2,figsize=(16,4))

ax[0].plot(learning_rates,accByLR,'s-')
ax[0].set_xlabel('Learning rate')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Accuracy by learning rate')

ax[1].plot(allLosses.T)
ax[1].set_title('Losses by learning rate')
ax[1].set_xlabel('Epoch number')
ax[1].set_ylabel('Loss')
plt.show()

In [ ]:
accByLR

In [ ]:
sum(torch.tensor(accByLR)>70)/len(accByLR)